# Challenge 4 study - Bruno

## Import libraries and data, init virtual device

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D

import akida
from akida import Device, NSoC_v2
import cnn2snn

device = akida.AKD1000()

2022-11-10 08:32:04.327578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 08:32:04.473238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 08:32:04.473264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 08:32:04.505690: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 08:32:05.331316: W tensorflow/stream_executor/platform/de

In [2]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

## Base model performances evaluation

### Performance of the "classic" keras model using the unprocessed input

In [3]:
base = Sequential(name="Base_ch4")
base.add(Dense(1000, input_shape=(2,), name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, input_shape=(2,), name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, input_shape=(2,), name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, input_shape=(2,), name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, input_shape=(2,), name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, input_shape=(2,), name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

In [4]:
base.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
init_weights = base.get_weights()

base.fit(coords_ch4, color, batch_size=64, epochs=200)
base.evaluate(coords_ch4, color)

Epoch 1/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6935 - accuracy: 0.5008
Epoch 2/200
157/157 [==============================] - 2s 12ms/step - loss: 0.6933 - accuracy: 0.4976
Epoch 3/200
157/157 [==============================] - 2s 12ms/step - loss: 0.6933 - accuracy: 0.5047
Epoch 4/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6929 - accuracy: 0.5100
Epoch 5/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6929 - accuracy: 0.5035
Epoch 6/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6926 - accuracy: 0.4999
Epoch 7/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6930 - accuracy: 0.4998
Epoch 8/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6932 - accuracy: 0.5057
Epoch 9/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6932 - accuracy: 0.5018
Epoch 10/200
157/157 [==============================] - 2s 12ms/step - loss: 0.693

157/157 [==============================] - 2s 11ms/step - loss: 0.1042 - accuracy: 0.9575
Epoch 81/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0721 - accuracy: 0.9681
Epoch 82/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0718 - accuracy: 0.9708
Epoch 83/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0876 - accuracy: 0.9616
Epoch 84/200
157/157 [==============================] - 2s 11ms/step - loss: 0.1136 - accuracy: 0.9542
Epoch 85/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0685 - accuracy: 0.9716
Epoch 86/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0774 - accuracy: 0.9677
Epoch 87/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0988 - accuracy: 0.9570
Epoch 88/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0891 - accuracy: 0.9616
Epoch 89/200
157/157 [==============================] - 2s 12ms/step - loss: 0.1105 - 

157/157 [==============================] - 2s 12ms/step - loss: 0.0850 - accuracy: 0.9637
Epoch 160/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0840 - accuracy: 0.9648
Epoch 161/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0647 - accuracy: 0.9714
Epoch 162/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0715 - accuracy: 0.9703
Epoch 163/200
157/157 [==============================] - 2s 12ms/step - loss: 0.0734 - accuracy: 0.9685
Epoch 164/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0791 - accuracy: 0.9655
Epoch 165/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0809 - accuracy: 0.9636
Epoch 166/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0794 - accuracy: 0.9654
Epoch 167/200
157/157 [==============================] - 2s 11ms/step - loss: 0.0643 - accuracy: 0.9722
Epoch 168/200
157/157 [==============================] - 2s 12ms/step - loss: 

[0.08657214790582657, 0.9592000246047974]

In [5]:
base_weights = base.get_weights()

### Input Coordinates Quantization on 4 bits and type conversion

In [16]:
qcoords = coords_ch4 * 15

# Following conversion if for use with model quantized on 4 bits
qcoords_u8 = qcoords.astype(np.uint8)

# Following conversion is to check performance of the base model with quantized coords (taken back between 0 and 1)
qcoords = qcoords.round() / 15
qcoords

array([[0.86666667, 0.53333333],
       [0.66666667, 0.26666667],
       [0.46666667, 0.6       ],
       ...,
       [0.33333333, 0.46666667],
       [0.4       , 0.93333333],
       [0.26666667, 0.33333333]])

### Evaluate the effect of input quantization on performace of the base model

In [7]:
base.evaluate(qcoords, color)

313/313 [==============================] - 1s 4ms/step - loss: 0.7353 - accuracy: 0.8619


[0.735325813293457, 0.8618999719619751]

### Retrain base model using quantized coordinates

In [8]:
base.set_weights(init_weights) # reinit
base.fit(qcoords, color, batch_size=64, epochs=200)
base.evaluate(qcoords, color)

Epoch 1/200
157/157 [==============================] - 2s 12ms/step - loss: 0.6939 - accuracy: 0.4983
Epoch 2/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6925 - accuracy: 0.5077
Epoch 3/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6900 - accuracy: 0.5097
Epoch 4/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6878 - accuracy: 0.5181
Epoch 5/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6693 - accuracy: 0.5438
Epoch 6/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6616 - accuracy: 0.5627
Epoch 7/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6471 - accuracy: 0.5748
Epoch 8/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6280 - accuracy: 0.5835
Epoch 9/200
157/157 [==============================] - 2s 11ms/step - loss: 0.6017 - accuracy: 0.5976
Epoch 10/200
157/157 [==============================] - 2s 11ms/step - loss: 0.584

157/157 [==============================] - 2s 11ms/step - loss: 0.3005 - accuracy: 0.8552
Epoch 81/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3033 - accuracy: 0.8578
Epoch 82/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3017 - accuracy: 0.8566
Epoch 83/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3014 - accuracy: 0.8576
Epoch 84/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3015 - accuracy: 0.8592
Epoch 85/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3073 - accuracy: 0.8546
Epoch 86/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3018 - accuracy: 0.8565
Epoch 87/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3006 - accuracy: 0.8553
Epoch 88/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3005 - accuracy: 0.8557
Epoch 89/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3032 - 

157/157 [==============================] - 2s 11ms/step - loss: 0.2986 - accuracy: 0.8565
Epoch 160/200
157/157 [==============================] - 2s 11ms/step - loss: 0.2969 - accuracy: 0.8583
Epoch 161/200
157/157 [==============================] - 2s 11ms/step - loss: 0.2956 - accuracy: 0.8594
Epoch 162/200
157/157 [==============================] - 2s 11ms/step - loss: 0.3023 - accuracy: 0.8565
Epoch 163/200
157/157 [==============================] - 2s 11ms/step - loss: 0.2926 - accuracy: 0.8579
Epoch 164/200
157/157 [==============================] - 2s 11ms/step - loss: 0.2949 - accuracy: 0.8555
Epoch 165/200
157/157 [==============================] - 2s 12ms/step - loss: 0.3045 - accuracy: 0.8542
Epoch 166/200
157/157 [==============================] - 2s 10ms/step - loss: 0.2966 - accuracy: 0.8565
Epoch 167/200
157/157 [==============================] - 2s 11ms/step - loss: 0.2956 - accuracy: 0.8572
Epoch 168/200
157/157 [==============================] - 2s 11ms/step - loss: 

[0.28400322794914246, 0.8652999997138977]

In [9]:
qbase_weights = base.get_weights()

## Model quantization

In [23]:
base.set_weights(base_weights) # reset values from first classic training
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4, input_weight_quantization=8)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=False))

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 FC1 (QuantizedDense)        (None, 1000)              3000      
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)               500500    
                                                                 
 relu2 (QuantizedReLU)       (None, 500)               0         
                                                                 
 FC3 (QuantizedDense)        (None, 250)               125250    
                                                                 
 relu3 (QuantizedReLU)       (None, 250)               0         
                                                                 
 FC4 (QuantizedDense)        (None, 100)             

In [26]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(qcoords_u8, color, batch_size=64, epochs=200)
qmodel.evaluate(qcoords_u8, color)

Epoch 1/200
157/157 [==============================] - 7s 32ms/step - loss: 5.9917 - accuracy: 0.5290
Epoch 2/200
157/157 [==============================] - 5s 29ms/step - loss: 0.8251 - accuracy: 0.5592
Epoch 3/200
157/157 [==============================] - 5s 31ms/step - loss: 0.7422 - accuracy: 0.5831
Epoch 4/200
157/157 [==============================] - 5s 32ms/step - loss: 0.7741 - accuracy: 0.6011
Epoch 5/200
157/157 [==============================] - 5s 30ms/step - loss: 0.7143 - accuracy: 0.6162
Epoch 6/200
157/157 [==============================] - 5s 30ms/step - loss: 0.7920 - accuracy: 0.5715
Epoch 7/200
157/157 [==============================] - 5s 30ms/step - loss: 0.7176 - accuracy: 0.5932
Epoch 8/200
157/157 [==============================] - 5s 30ms/step - loss: 0.6593 - accuracy: 0.6329
Epoch 9/200
157/157 [==============================] - 5s 30ms/step - loss: 0.6538 - accuracy: 0.6626
Epoch 10/200
157/157 [==============================] - 5s 30ms/step - loss: 0.630

157/157 [==============================] - 5s 29ms/step - loss: 0.3789 - accuracy: 0.8321
Epoch 81/200
157/157 [==============================] - 5s 30ms/step - loss: 0.3801 - accuracy: 0.8319
Epoch 82/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3686 - accuracy: 0.8348
Epoch 83/200
157/157 [==============================] - 5s 32ms/step - loss: 0.3661 - accuracy: 0.8364
Epoch 84/200
157/157 [==============================] - 5s 31ms/step - loss: 0.3737 - accuracy: 0.8337
Epoch 85/200
157/157 [==============================] - 5s 32ms/step - loss: 0.3680 - accuracy: 0.8368
Epoch 86/200
157/157 [==============================] - 6s 36ms/step - loss: 0.3686 - accuracy: 0.8364
Epoch 87/200
157/157 [==============================] - 5s 32ms/step - loss: 0.3675 - accuracy: 0.8334
Epoch 88/200
157/157 [==============================] - 5s 33ms/step - loss: 0.3632 - accuracy: 0.8377
Epoch 89/200
157/157 [==============================] - 5s 30ms/step - loss: 0.3699 - 

157/157 [==============================] - 5s 29ms/step - loss: 0.3574 - accuracy: 0.8390
Epoch 160/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3525 - accuracy: 0.8409
Epoch 161/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3561 - accuracy: 0.8384
Epoch 162/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3549 - accuracy: 0.8443
Epoch 163/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3539 - accuracy: 0.8424
Epoch 164/200
157/157 [==============================] - 5s 30ms/step - loss: 0.3519 - accuracy: 0.8445
Epoch 165/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3413 - accuracy: 0.8462
Epoch 166/200
157/157 [==============================] - 5s 32ms/step - loss: 0.3415 - accuracy: 0.8491
Epoch 167/200
157/157 [==============================] - 5s 29ms/step - loss: 0.3529 - accuracy: 0.8414
Epoch 168/200
157/157 [==============================] - 5s 29ms/step - loss: 

[0.34084418416023254, 0.8468000292778015]

In [24]:
akmodel = cnn2snn.convert(qmodel, input_is_image=False)
akmodel.summary()
akmodel.map(device)


                Model Summary                 
______________________________________________
Input shape  Output shape  Sequences  Layers
[1, 1, 2]    [1, 1, 1]     1          7     
______________________________________________

_______________________________________________
Layer (type)  Output shape  Kernel shape     

============ SW/FC1-FC6 (Software) ============

FC1 (Fully.)  [1, 1, 1000]  (1, 1, 2, 1000)  
_______________________________________________
FC2 (Fully.)  [1, 1, 500]   (1, 1, 1000, 500)
_______________________________________________
FC3 (Fully.)  [1, 1, 250]   (1, 1, 500, 250) 
_______________________________________________
FC4 (Fully.)  [1, 1, 100]   (1, 1, 250, 100) 
_______________________________________________
FC5 (Fully.)  [1, 1, 50]    (1, 1, 100, 50)  
_______________________________________________
FC6 (Fully.)  [1, 1, 1]     (1, 1, 50, 1)    
_______________________________________________


In [25]:
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[1, 1, 2]    [1, 1, 1]     2          7       5  
___________________________________________________

____________________________________________________
Layer (type)  Output shape  Kernel shape       NPs

================= SW/FC1 (Software) ================

FC1 (Fully.)  [1, 1, 1000]  (1, 1, 2, 1000)    N/A

==== HW/FC2-FC6 (Hardware) - size: 520116 bytes ====

FC2 (Fully.)  [1, 1, 500]   (1, 1, 1000, 500)  1  
____________________________________________________
FC3 (Fully.)  [1, 1, 250]   (1, 1, 500, 250)   1  
____________________________________________________
FC4 (Fully.)  [1, 1, 100]   (1, 1, 250, 100)   1  
____________________________________________________
FC5 (Fully.)  [1, 1, 50]    (1, 1, 100, 50)    1  
____________________________________________________
FC6 (Fully.)  [1, 1, 1]     (1, 1, 50, 1)      1  
___________